In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows', 300)  # Show all rows

os.chdir('/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Machine_Learning_Model/Defence_Profit_Loss_calculated/')
# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Machine_Learning_Model/Defence_Profit_Loss_calculated/'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['trade_data_defence_profit_loss_data.csv',
 'ideal_data_defence_profit_loss_data.csv',
 'cal_data_defence_profit_loss_data (1).csv']

In [5]:
trade_data = pd.read_csv("trade_data_defence_profit_loss_data.csv")
ideal_data = pd.read_csv("ideal_data_defence_profit_loss_data.csv")
cal_data = pd.read_csv("cal_data_defence_profit_loss_data (1).csv")

In [6]:
trade_data.shape , ideal_data.shape , cal_data.shape

((505221, 29), (505221, 29), (505221, 29))

In [7]:
trade_data.drop('Unnamed: 0',axis=1,inplace=True)
ideal_data.drop('Unnamed: 0',axis=1,inplace=True)
cal_data.drop('Unnamed: 0',axis=1,inplace=True)
trade_data["Time Stamp"] = pd.to_datetime(trade_data["Time Stamp"])
ideal_data["Time Stamp"] = pd.to_datetime(ideal_data["Time Stamp"])
cal_data["Time Stamp"] = pd.to_datetime(cal_data["Time Stamp"])

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [9]:
# Assuming cal_data is already defined and processed
cal_data.drop(['BuyersInControlTime', 'SellersInControlTime', 'Profit_C', 'Loss_C', 'D_Profit_C', 'D_Loss_C'], axis=1, inplace=True)
cal_data.set_index('Time Stamp', inplace=True)

In [10]:
# Split the data
train_data = cal_data[cal_data.index < '2023-06-23 00:05:00']
test_data = cal_data[cal_data.index >= '2023-06-24 00:05:00']

In [11]:
# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(train_data)

In [12]:
# Preparing the sequences
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        target = data[i + sequence_length, -4:]  # Assuming last 4 columns are the targets
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

In [13]:
# Set the sequence length
sequence_length = 60  # Adjust this as needed
X_train, y_train = create_sequences(scaled_data, sequence_length)

In [14]:

# Define the number of features
n_features = X_train.shape[2]

In [15]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(sequence_length, n_features)))
model.add(Dropout(0.2))
model.add(Dense(4))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [17]:
# Display the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 100)                 │          48,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │             404 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,204 (192.20 KB)

 Trainable params: 49,204 (192.20 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
12642/12642 ━━━━━━━━━━━━━━━━━━━━ 580s 46ms/step - loss: 0.1151
Epoch 2/10
12642/12642 ━━━━━━━━━━━━━━━━━━━━ 618s 45ms/step - loss: 0.1114
Epoch 3/10
12642/12642 ━━━━━━━━━━━━━━━━━━━━ 616s 45ms/step - loss: 0.1113
Epoch 4/10
12642/12642 ━━━━━━━━━━━━━━━━━━━━ 627s 45ms/step - loss: 0.1114
Epoch 5/10
12642/12642 ━━━━━━━━━━━━━━━━━━━━ 622s 45ms/step - loss: 0.1110
Epoch 6/10
12642/12642 ━━━━━━━━━━━━━━━━━━━━ 615s 45ms/step - loss: 0.1110
Epoch 7/10
 9189/12642 ━━━━━━━━━━━━━━━━━━━━ 2:30 44ms/step - loss: 0.1108

In [ ]:
# Prepare test data
scaled_test_data = scaler.transform(test_data)
X_test, y_test = create_sequences(scaled_test_data, sequence_length)

In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Rescale predictions to original scale
predictions_rescaled = scaler.inverse_transform(predictions)

In [ ]:
# Print predictions
print(predictions_rescaled)